# Setup Gym Environment

```py
my_config["interface"] = MyGranTurismoRTGYM
my_config["time_step_duration"] = 0.05 # when to give up
my_config["start_obs_capture"] = 0.05 # when to capture 
my_config["time_step_timeout_factor"] = 1.0 # how late is OK
my_config["act_buf_len"] = 3 # how many past actions
my_config["reset_act_buf"] = True # resect action buffer on reset
my_config["benchmark"] = True
my_config["benchmark_polyak"] = 0.2
```

This section needs to be setup for any method

In [4]:
from myRTClass import MyGranTurismoRTGYM, DEFAULT_CONFIG_DICT
import gymnasium

my_config = DEFAULT_CONFIG_DICT
my_config["interface"] = MyGranTurismoRTGYM
my_config["time_step_duration"] = 0.05
my_config["start_obs_capture"] = 0.05
my_config["time_step_timeout_factor"] = 1.0
my_config["ep_max_length"] = 2048
my_config["act_buf_len"] = 3
my_config["reset_act_buf"] = False
my_config["benchmark"] = True
my_config["benchmark_polyak"] = 0.2

my_config["interface_kwargs"] = {
  'debugFlag': False, # do not use render() while True
  'img_hist_len': 3,
  'modelMode': 3,
  'agent' : 'PPO',
  #  [42, 42, K], [84, 84, K], [10, 10, K], [240, 320, K] and  [480, 640, K]
  'imageWidth' : 42, # there is a default Cov layer for PPO with 240 x 320
  'imageHeight' : 42,
  'trackChoice' : 1, # 1 is High Speed Ring, # 2 is 0-400m
}

SyntaxError: invalid syntax (4216221866.py, line 1)

In [2]:
env = gymnasium.make("real-time-gym-v1", config=my_config)

# Register the environment in a way that RLlib is happy

In [7]:
if not debugAsGym:
    def env_creator(env_config):
        env = gymnasium.make("real-time-gym-v1", config=env_config)
        return env  # return an env instance

    from ray.tune.registry import register_env
    register_env("gt-rtgym-env-v1", lambda config: env_creator(my_config)) # better way

In [8]:
if not debugAsGym:
    import ray
    ray.shutdown()
    ray.init()

2023-05-15 00:01:23,516	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


In [9]:
if not debugAsGym:
    from ray.rllib.algorithms.ppo import PPOConfig

    algo = (
        PPOConfig()
        .resources(
            num_gpus=1
            )
        .rollouts(
            num_rollout_workers=1,
            enable_connectors=True,
            batch_mode="truncate_episodes",
            )
        .framework("torch")
        .environment(
            env="gt-rtgym-env-v1",
            disable_env_checking=True,
            render_env=False,
            )
        .training(
            #lr=tune.grid_search([0.01, 0.001, 0.0001])
            #lambda_=0.95,
            #gamma=0.99,
            #sgd_minibatch_size=128,
            train_batch_size=256,
            #num_sgd_iter=8,
            #clip_param=0.2,
            model={"fcnet_hiddens": [8, 8]},
        )
        .build()
    )

2023-05-15 00:01:28,966	INFO algorithm.py:506 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


(RolloutWorker pid=12448) GT Real Time instantiated
(RolloutWorker pid=12448) GT AI Server instantiated for rtgym
(RolloutWorker pid=12448) starting up on localhost port 9999
(RolloutWorker pid=12448) Waiting for a connection
(RolloutWorker pid=12448) Connection from ('127.0.0.1', 52653)


2023-05-15 00:01:42,896	INFO trainable.py:172 -- Trainable.setup took 13.931 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2023-05-15 00:01:42,898	WARNING util.py:67 -- Install gputil for GPU system monitoring.


In [10]:
if not debugAsGym:
    N = 10000

    for n in range(N):
        result = algo.train()
        print("Loop: ", n)
        if n % 10 == 0:
            print("Saved", n)
            algo.save()
            
    algo.save()


(RolloutWorker pid=12448) reset triggered
(RolloutWorker pid=12448) reload save for track : 2
Loop:  0
Saved 0


(RolloutWorker pid=12448) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 4.552719199999817
(RolloutWorker pid=12448)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


Loop:  1


(RolloutWorker pid=12448) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 4.449114200000686
(RolloutWorker pid=12448)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


Loop:  2


...

(RolloutWorker pid=12448) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 4.34695129989268
(RolloutWorker pid=12448)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


(RolloutWorker pid=12448) reset triggered
(RolloutWorker pid=12448) reload save for track : 2


(RolloutWorker pid=12448) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.15439899926423095
(RolloutWorker pid=12448)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


Loop:  2647


(RolloutWorker pid=12448) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 4.353202599988435
(RolloutWorker pid=12448)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
